<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/commons-io-master/cio_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
save_file_csv = 'commons-io-master_class_all.csv'
file_csv =  'commons-io-master.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/commons-io-master' #Newly stored class address

In [7]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [8]:
fileList = []
file_name(file_dir)

In [9]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}


In [10]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #排除汉字
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #排除中文符号
    #print("unchinese:",unchinese)
    return unchinese

In [11]:
def del_stopwords(line):
#     for line in file.readlines():
#     dicts = {i: '' for i in punctuation}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

#     for word in stopwords:
#           new_line = new_line.replace(word, '')

    return new_line

In [12]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)

T
T


In [13]:
Brain_Class = np.array(Brain_Class)
len(Brain_Class)

352

In [14]:
len(index)

352

In [15]:
#??????????????????????????????????
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [16]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['DemuxInputStream', 'Binds', 'the', 'specified', 'stream', 'to', 'the', 'current', 'param', 'input', 'the', 'stream', 'to', 'return', 'the', 'InputStream', 'that', 'was', 'previously', 'InputStream', 'bindStreamfinal', 'InputStream', 'final', 'InputStream', 'return'])
 list(['CanWriteFileFilter', 'Restrictive'])
 list(['CollectionFileListener', 'Clear', 'file', 'void', 'Create', 'a', 'new', 'param', 'clearOnStart', 'true', 'if', 'clear', 'should', 'be', 'called', 'by', 'CollectionFileListenerfinal', 'boolean', 'Return', 'the', 'set', 'of', 'changed', 'return', 'Directories', 'which', 'have', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of', 'changed', 'return', 'Files', 'which', 'have', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of', 'created', 'return', 'Directories', 'which', 'have', 'been', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of', 'created', 'return', 'Files', 'which', 'have', 'been', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of',

In [17]:
# #training phase
def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    # print(vectorizer.get_feature_names())#Get the vocabulary found by the model directly analyzing the data（collection of words above）
    # print(X.toarray())# Printing X directly outputs the position of each word
    #     print(X)

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  # Latent semantic analysis, set 4 topics
    X2 = svd_model.fit_transform(X)  # train and convert

    # print("--------lsa Singular Value---------")
    # print(svd_model.singular_values_)
    # print("--------Representation of text in topic vector space Left singular matrix---------")
    # print(X2)

    terms = vectorizer.get_feature_names()
    # print("-------- Topics and keywords ---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [18]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['DemuxInputStream', 'Binds', 'the', 'specified', 'stream', 'to', 'the', 'current', 'param', 'input', 'the', 'stream', 'to', 'return', 'the', 'InputStream', 'that', 'was', 'previously', 'InputStream', 'bindStreamfinal', 'InputStream', 'final', 'InputStream', 'return'] ****************
inputstream
bindstreamfinal
demuxinputstream
binds
that
specified
previously
 
************* ['CanWriteFileFilter', 'Restrictive'] ****************
canwritefilefilter
restrictive
 
************* ['CollectionFileListener', 'Clear', 'file', 'void', 'Create', 'a', 'new', 'param', 'clearOnStart', 'true', 'if', 'clear', 'should', 'be', 'called', 'by', 'CollectionFileListenerfinal', 'boolean', 'Return', 'the', 'set', 'of', 'changed', 'return', 'Directories', 'which', 'have', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of', 'changed', 'return', 'Files', 'which', 'have', 'CollectionFile', 'return', 'Return', 'the', 'set', 'of', 'created', 'return', 'Directories', 'which', 'have', 'been', 'C

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


************* ['FileAlterationObserver (1)', 'Adds', 'a', 'file', 'system', 'param', 'listener', 'The', 'file', 'system', 'void', 'addListenerfinal', 'FileAlterationListener', 'if', 'listener', 'null', 'Compares', 'two', 'file', 'lists', 'for', 'files', 'which', 'have', 'been', 'created', 'modified', 'or', 'param', 'parent', 'The', 'parent', 'param', 'previous', 'The', 'original', 'list', 'of', 'param', 'files', 'The', 'current', 'list', 'of', 'void', 'checkAndNotifyfinal', 'FileEntry', 'parentfinal', 'FileEntry', 'previousfinal', 'File', 'int', 'final', 'FileEntry', 'currentfileslength', '0', 'new', 'FileEntryfileslength', 'for', 'final', 'FileEntry', 'entry', 'previous', 'while', 'c', 'fileslength', 'comparatorcompareentrygetFilefilesc', '0', 'if', 'c', 'fileslength', 'comparatorcompareentrygetFilefilesc', '0', 'else', 'for', 'c', 'fileslength', 'c', 'Creates', 'a', 'new', 'file', 'entry', 'for', 'the', 'specified', 'param', 'parent', 'The', 'parent', 'file', 'param', 'file', 'The', 

In [19]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

c = np.array(c)


In [20]:
from keras import Input,Model
from keras.layers import Dense

In [21]:
def word2vec_train(index):
    list = []
    list.append(index[0])
    words = []

    additem(index)
    # for text in index:
    #     #     print(text)
    print(index)
    for word in index:
        if word not in words:
            words.append(word)

    # words = set(words)
    # print(words)

    word2int = {}
    for i,word in enumerate(words):
        word2int[word] = i
    print(word2int)



    WINDOW_SIZE=2
    data = []
    # for sentence in index:
    for idx,word in enumerate(index):
        for neighbor in index[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(index))+ 1]:
            if neighbor != word:
                data.append([word, neighbor])
    #                 print(data)
    df = pd.DataFrame(data,columns=['input','label'])

    # print(df)

    ONE_HOT_DIM = len(words)
    #function to convert numbers to one hot vectors
    def to_one_hot_encoding(data_point_index):
        one_hot_encoding = np.zeros(ONE_HOT_DIM)
        one_hot_encoding[data_point_index] = 1
        return one_hot_encoding

    X = []
    Y = []

    for x, y in zip(df['input'],df['label']):
        X.append(to_one_hot_encoding(word2int[x]))
        Y.append(to_one_hot_encoding(word2int[y]))

    x_train = np.array(X)
    y_train = np.array(Y)


    #Defining the size of the embedding
    embed_size = 3
    xx = Input(shape=(x_train.shape[1],))
    yy = Dense(units=embed_size, activation='linear')(xx)
    yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
    model = Model(inputs=xx, outputs=yy)
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
    #optimizing the network weights
    model.fit(x=x_train,y=y_train,
            batch_size=128,
            epochs=5
    )
    weights = model.get_weights()[0]
    # embedding = []
    # embedding.append[index[0]]

    for word in words:
        # list = []
        # list.append(word)
        q = weights[df.get(word)]

        arr_new = np.sum(q, axis=1)
        print('-----------------------')
        print(arr_new.shape)
        print('-----------------------')
        arr_new = arr_new.tolist()

        list.append(arr_new)

        break
        # list.append(q)
        # # print(weights[df.get(word)].shape)
        # print(word)
        # print(q)
        # print('---')
        # embedding.append(list)
    return list

In [22]:
embedding = []
for item in index:
    embedding.append(word2vec_train(item))
print(embedding)

Streaming output truncated to the last 5000 lines.
11/11 [==============================] - 0s 2ms/step - loss: 4.4839
-----------------------
(1, 3)
-----------------------
['FileExistsException', 'Default', 'Constructs', 'an', 'instance', 'with', 'the', 'specified', 'param', 'file', 'The', 'file', 'that', 'FileExistsExceptionfinal', 'File', 'superFile', 'file', 'Constructs', 'an', 'instance', 'with', 'the', 'specified', 'param', 'message', 'The', 'error', 'FileExistsExceptionfinal', 'String']
{'FileExistsException': 0, 'Default': 1, 'Constructs': 2, 'an': 3, 'instance': 4, 'with': 5, 'the': 6, 'specified': 7, 'param': 8, 'file': 9, 'The': 10, 'that': 11, 'FileExistsExceptionfinal': 12, 'File': 13, 'superFile': 14, 'message': 15, 'error': 16, 'String': 17}
Epoch 1/5
1/1 [==============================] - 0s 330ms/step - loss: 2.8589
Epoch 2/5
1/1 [==============================] - 0s 9ms/step - loss: 2.8579
Epoch 3/5
1/1 [==============================] - 0s 8ms/step - loss: 2.8569
Ep

In [23]:
List = []
len(embedding)

352

In [26]:
for i in range(0,len(embedding)):
    st = str(embedding[i][0])
    st = st[1:]
#     print(st)

            # embedding[i][1].reshape((1,3))
            # l = embedding[i][1].tolist()
    List.append(embedding[i][1])

In [27]:
List

[[[0.7003371715545654, 1.0071393251419067, 0.6224311590194702]],
 [[1.067895531654358, -2.0447049140930176, -0.2813519835472107]],
 [[0.23227141797542572, 0.1394662708044052, -0.3986741900444031]],
 [[1.3890955448150635, 2.292659282684326, -2.236299514770508]],
 [[0.4675716459751129, -0.24880656599998474, -0.883724570274353]],
 [[0.7152122855186462, 2.228076934814453, 1.3479453325271606]],
 [[-1.10308837890625, 1.903598427772522, 0.41383829712867737]],
 [[-0.07145333290100098, 0.5780829191207886, 0.3592832386493683]],
 [[0.3449276089668274, -0.5279818177223206, 2.5522401332855225]],
 [[-0.7215075492858887, -0.35517576336860657, -0.44225767254829407]],
 [[-0.2805018424987793, 0.436147540807724, -0.6158055067062378]],
 [[-0.2678687572479248, 1.3664952516555786, 0.4079824388027191]],
 [[-0.36998236179351807, -1.76235830783844, 2.59499454498291]],
 [[0.32643312215805054, -0.9237639904022217, -0.3398248553276062]],
 [[0.033125005662441254, -1.6390475034713745, -1.8804137706756592]],
 [[2.38